# Chapter 3: Convolutional Neural Networks
You'll need to either copy the image training files you downloaded in Chapter 2 to this directory, or alter the paths appropriately.

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data
import torch.nn.functional as F
import torchvision
from torchvision import transforms
from PIL import Image

## CNNNet (or AlexNet by another name…)

In [2]:
class CNNNet(nn.Module):

    def __init__(self, num_classes=2):
        super(CNNNet, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=11, stride=4, padding=2),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Conv2d(64, 192, kernel_size=5, padding=2),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Conv2d(192, 384, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(384, 256, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=3, stride=2),
        )
        self.avgpool = nn.AdaptiveAvgPool2d((6, 6))
        self.classifier = nn.Sequential(
            nn.Dropout(),
            nn.Linear(256 * 6 * 6, 4096),
            nn.ReLU(),
            nn.Dropout(),
            nn.Linear(4096, 4096),
            nn.ReLU(),
            nn.Linear(4096, num_classes)
        )

    def forward(self, x):
        x = self.features(x)
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        return x

In [3]:
cnnnet = CNNNet()

In [4]:
def train(model, optimizer, loss_fn, train_loader, val_loader, epochs=20, device="cpu"):
    for epoch in range(1, epochs+1):
        training_loss = 0.0
        valid_loss = 0.0
        model.train()
        for batch in train_loader:
            optimizer.zero_grad()
            inputs, targets = batch
            inputs = inputs.to(device)
            targets = targets.to(device)
            output = model(inputs)
            loss = loss_fn(output, targets)
            loss.backward()
            optimizer.step()
            training_loss += loss.data.item() * inputs.size(0)
        training_loss /= len(train_loader.dataset)

        model.eval()
        num_correct = 0
        num_examples = 0
        for batch in val_loader:
            inputs, targets = batch
            inputs = inputs.to(device)
            output = model(inputs)
            targets = targets.to(device)
            loss = loss_fn(output,targets)
            valid_loss += loss.data.item() * inputs.size(0)
            correct = torch.eq(torch.max(F.softmax(output, dim=1), dim=1)[1],
                               targets)
            num_correct += torch.sum(correct).item()
            num_examples += correct.shape[0]
        valid_loss /= len(val_loader.dataset)

        print('Epoch: {}, Training Loss: {:.2f}, Validation Loss: {:.2f}, accuracy = {:.2f}'.format(epoch, training_loss,
        valid_loss, num_correct / num_examples))

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [9]:
import os
import zipfile
import shutil
import random

# 1. 환경 설정
base_path = '/content'
categories = ['cat', 'fish']
splits = ['train', 'val', 'test']
zip_path ='/content/drive/MyDrive/Deep Learning/beginners-pytorch-deep-learning-master/chapter2_Image Classification with PyTorch/images.zip'
temp_extract_path = '/content/temp_images'

# 2. 최종 목적지 디렉토리 생성
for s in splits:
    for c in categories:
        os.makedirs(os.path.join(base_path, s, c), exist_ok=True)

# 3. 압축 해제
zip_file_extracted = False
if os.path.exists(zip_path):
    # 기존 임시 폴더가 있으면 삭제 후 생성
    if os.path.exists(temp_extract_path):
        shutil.rmtree(temp_extract_path)

    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(temp_extract_path)
    print("✅ 1. 압축 해제 완료")
    zip_file_extracted = True
else:
    print(f"❌ Error: {zip_path} 파일을 찾을 수 없습니다.")

# 4. 실제 cat, fish 폴더의 위치를 찾는 함수
def find_category_path(root_path, target_category):
    for root, dirs, files in os.walk(root_path):
        if target_category in dirs:
            return os.path.join(root, target_category)
    return None

# 5. 폴더 기반 분류 및 이동 (zip 파일이 성공적으로 추출된 경우에만 진행)
if zip_file_extracted:
    for category in categories:
        # 임시 폴더 내부 어디든 'cat' 또는 'fish' 폴더가 있는지 검색
        src_category_dir = find_category_path(temp_extract_path, category)

        if src_category_dir is None:
            print(f"⚠️ 경고: 압축 파일 내에서 '{category}' 폴더를 찾을 수 없습니다.")
            continue
        else:
            print(f"📂 '{category}' 폴더 발견: {src_category_dir}")

        # 해당 폴더 내의 이미지 파일 목록 가져오기
        all_files = [f for f in os.listdir(src_category_dir)
                     if os.path.isfile(os.path.join(src_category_dir, f)) and not f.startswith('.')]

        random.shuffle(all_files)

        # 분할 비율 (8:1:1)
        train_idx = int(len(all_files) * 0.8)
        val_idx = int(len(all_files) * 0.9)

        # 파일 이동
        for i, f in enumerate(all_files):
            src_path = os.path.join(src_category_dir, f)
            if i < train_idx:
                dst_path = os.path.join(base_path, 'train', category, f)
            elif i < val_idx:
                dst_path = os.path.join(base_path, 'val', category, f)
            else:
                dst_path = os.path.join(base_path, 'test', category, f)
            shutil.move(src_path, dst_path)

# 6. 임시 폴더 삭제
if os.path.exists(temp_extract_path):
    shutil.rmtree(temp_extract_path)
print("\n✨ 모든 분류 및 이동 작업이 완료되었습니다!")

# 7. 결과 확인
print("-" * 30)
for s in splits:
    for c in categories:
        count = len(os.listdir(os.path.join(base_path, s, c)))
        print(f"✅ {s}/{c} 폴더 파일 수: {count}")

✅ 1. 압축 해제 완료
📂 'cat' 폴더 발견: /content/temp_images/val/cat
📂 'fish' 폴더 발견: /content/temp_images/val/fish

✨ 모든 분류 및 이동 작업이 완료되었습니다!
------------------------------
✅ train/cat 폴더 파일 수: 80
✅ train/fish 폴더 파일 수: 44
✅ val/cat 폴더 파일 수: 10
✅ val/fish 폴더 파일 수: 5
✅ test/cat 폴더 파일 수: 10
✅ test/fish 폴더 파일 수: 6


In [10]:
def check_image(path):
    try:
        im = Image.open(path)
        return True
    except:
        return False

img_transforms = transforms.Compose([
    transforms.Resize((64,64)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
    ])
train_data_path = "./train/"
train_data = torchvision.datasets.ImageFolder(root=train_data_path,transform=img_transforms, is_valid_file=check_image)
val_data_path = "./val/"
val_data = torchvision.datasets.ImageFolder(root=val_data_path,transform=img_transforms, is_valid_file=check_image)

batch_size=64
train_data_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size,shuffle=True)
val_data_loader  = torch.utils.data.DataLoader(val_data, batch_size=batch_size, shuffle=True)

if torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")

In [11]:
cnnnet.to(device)
optimizer = optim.Adam(cnnnet.parameters(), lr=0.001)

In [12]:
train(cnnnet, optimizer,torch.nn.CrossEntropyLoss(), train_data_loader,val_data_loader, epochs=10, device=device)

Epoch: 1, Training Loss: 2.87, Validation Loss: 0.54, accuracy = 0.80
Epoch: 2, Training Loss: 0.54, Validation Loss: 0.53, accuracy = 0.80
Epoch: 3, Training Loss: 0.52, Validation Loss: 0.51, accuracy = 0.80
Epoch: 4, Training Loss: 0.48, Validation Loss: 0.51, accuracy = 0.80
Epoch: 5, Training Loss: 0.47, Validation Loss: 0.63, accuracy = 0.80
Epoch: 6, Training Loss: 0.50, Validation Loss: 0.49, accuracy = 0.80
Epoch: 7, Training Loss: 0.46, Validation Loss: 0.55, accuracy = 0.80
Epoch: 8, Training Loss: 0.53, Validation Loss: 0.53, accuracy = 0.80
Epoch: 9, Training Loss: 0.51, Validation Loss: 0.50, accuracy = 0.80
Epoch: 10, Training Loss: 0.46, Validation Loss: 0.48, accuracy = 0.80


## Downloading a pretrained network

There are two ways of downloading pre-trained image models with PyTorch. Firstly, you can use the `torchvision.models` library, or you can use PyTorch Hub. The latter is preferred as of 2019, as this is a one-stop shop for all models and the new standard for distributing models with PyTorch.

In [13]:
import torchvision.models as models
alexnet = models.alexnet(num_classes=1000, pretrained=True)

/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Downloading: "https://download.pytorch.org/models/alexnet-owt-7be5be79.pth" to /root/.cache/torch/hub/checkpoints/alexnet-owt-7be5be79.pth


100%|██████████| 233M/233M [00:02<00:00, 99.1MB/s]


In [14]:
resnet50 = torch.hub.load('pytorch/vision', 'resnet50')

Downloading: "https://github.com/pytorch/vision/zipball/main" to /root/.cache/torch/hub/main.zip


In [15]:
print(alexnet)

AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (avgpool): AdaptiveAvgPool2d(output_size=(6, 6))
  (classifier): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=9216, out_features=4096, bias=True)
 

In [16]:
print(resnet50)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 